# Terrenos vs otro
limpieza al shp de terrenos.
- Identificar los terrenos de casas y apendarlos con terrenos
- Agrupar las geometrias 
- Usar el archivo cruces_limpieza_shp


In [1]:
# !pip install geopandas
import geopandas as gpd
import pandas as pd 
import sys
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import obtener_curt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mfpen\AppData\Local\Temp\ipykernel_208\1841221092.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedoc

In [2]:
def choose_city(gdf_zonas):
    '''
        Función para localizar el municipio a tratar dentro del shape de zonas rurales y urbanas
    '''
    zona = input('Digita el código de la zona que quieres (Ej. Ixtapan = 040):')
    gdf_mun = gdf_zonas[gdf_zonas.CVE_MUN == zona]
    #gdf_mun.plot()
    return gdf_mun
    
def cruce_limpieza_shapes(gdf_zonas,gdf_construcciones):
    gdf_mun = choose_city(gdf_zonas)
    gdf_mun = gdf_mun.to_crs(3857)
    gdf_cruce= gdf_construcciones.sjoin(gdf_mun,how='left')
    
    return gdf_cruce

In [81]:
# Usar el de casas original
path_casas = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapan_Sal\Ixtapan_completo\salida_original_y_nueva_rotacion\Ixtapan_casas_rotacion.shp"
# Usar terrenos original
path_terrenos = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapan_Sal\Ixtapan_completo\salida_original_y_nueva_rotacion\Ixtapan_Sal_terrenos.shp"
# Usar chinchetas original
path_chinchetas = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapan_Sal\Chinchetas\Ixtapan_Chinchetas_DNUE.shp"

In [82]:
casas = gpd.read_file(path_casas)
terrenos = gpd.read_file(path_terrenos)
pines = gpd.read_file(path_chinchetas)

In [83]:
terrenos.clase_dete.value_counts()

terreno_baldio     7598
establecimiento    1664
casas                56
Name: clase_dete, dtype: int64

In [84]:
casas['clase_dete'].value_counts()

establecimiento    20550
casas               7329
terreno_baldio      1329
Name: clase_dete, dtype: int64

In [85]:
# Terrenos de casas
new_terrenos = casas[casas['clase_dete']== 'terreno_baldio']
new_terrenos.shape

(1329, 5)

In [86]:
# Concatenar terrenos de casas a terrenos
terrenos = pd.concat([terrenos, new_terrenos])
terrenos.shape

(10647, 5)

In [87]:
# Etiquetar las casas y establecimientos
terrenos.loc[(terrenos['clase_dete']=='casas') | (terrenos['clase_dete']=='establecimiento'), 'con_const'] = 1
terrenos.loc[terrenos.con_const!=1, 'con_const'] = 0
terrenos.loc[(terrenos['clase_dete']=='casas') | (terrenos['clase_dete']=='establecimiento'), 'clase_dete'] = 'terreno_baldio'

In [88]:
terrenos=obtener_curt(terrenos,'geometry')
terrenos.tail(2)

C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos\codigos.py:347: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data['centroid'] = data[geom_col].to_crs(4326).centroid


,clase_dete,conf,area,geometry,status,con_const,CURT_f
29193,terreno_baldio,0.26252,244.368842,"POLYGON ((-11088769.944 2136447.412, -11088761...",correcto,0.0,18503401949936432164
29199,terreno_baldio,0.23584,225.261813,"POLYGON ((-11088707.717 2136449.590, -11088699...",correcto,0.0,18503404499936412337


In [89]:
# Revision de duplicados por geometria
terrenos.geometry.value_counts()

POLYGON ((-11105809.797 2135445.579, -11105744...    1
POLYGON ((-11105802.475 2133970.657, -11105745...    1
POLYGON ((-11105812.324 2134230.735, -11105751...    1
POLYGON ((-11105794.094 2135250.986, -11105743...    1
POLYGON ((-11105782.348 2135209.905, -11105744...    1
                                                    ..
POLYGON ((-11089076.812 2136684.389, -11089076...    1
POLYGON ((-11089044.751 2136773.933, -11089024...    1
POLYGON ((-11089033.483 2136406.426, -11089024...    1
POLYGON ((-11089005.477 2134903.035, -11088933...    1
POLYGON ((-11088707.717 2136449.590, -11088699...    1
Name: geometry, Length: 10647, dtype: int64

In [80]:
# Borrar duplicados
terrenos.sort_values('conf',ascending=False,inplace=True)
terrenos.drop_duplicates(subset='geometry',keep='first',inplace=True)
terrenos.tail(2)

,clase_dete,conf,area,geometry,status,con_const,CURT_f
1650,terreno_baldio,0.20004,1513.477810,"POLYGON ((-11101656.673 2136113.045, -11101630...",NaN,0.0,18502403769943394752
4504,terreno_baldio,0.20001,1128.320334,"POLYGON ((-11098159.496 2128633.781, -11098141...",NaN,1.0,18463436589941474224


In [90]:
pines.columns

Index(['Clase', 'tipoCenCom', 'tipo_asent', 'nombre_act', 'clee', 'id',
       'Clase1', 'Clase_lim', 'geometry'],
      dtype='object')

In [91]:
#Eliminar las columnas que traen el concatenado de CLase y solo dejar la columna buena
pines.drop(['Clase1','Clase'],axis=1,inplace=True)
pines.tail(2)

,tipoCenCom,tipo_asent,nombre_act,clee,id,Clase_lim,geometry
1814,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088585.525 2131199.662, -11088586..."
1815,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088485.810 2139040.121, -11088476..."


In [92]:
#La columna de clasificacion de chinchetas debe llamarse Clase, sino cambiar
pines.rename(columns={'Clase_lim':'Clase'},inplace=True)
pines.tail(2)

,tipoCenCom,tipo_asent,nombre_act,clee,id,Clase,geometry
1814,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088585.525 2131199.662, -11088586..."
1815,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088485.810 2139040.121, -11088476..."


In [93]:
#En el caso de ya tener pines vs denue, cambiar el nombre de la columna id a ID_denue
pines.rename(columns={'id':'ID_denue'},inplace=True)
pines.tail(2)

,tipoCenCom,tipo_asent,nombre_act,clee,ID_denue,Clase,geometry
1814,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088585.525 2131199.662, -11088586..."
1815,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088485.810 2139040.121, -11088476..."


In [94]:
pines.Clase.value_counts()

a_verdes                     407
tiendas_1                    358
restaurante_google           227
hospedaje_google              69
tiendas_2                     36
a_verdes_1                    28
bar                           22
a_verdes_2                     5
establecimientos_2_google      4
Hospital                       2
Name: Clase, dtype: int64

In [95]:
# Areas verdes de pines
pines_verdes = pines[(pines['Clase']== 'a_verdes')|(pines['Clase']== 'a_verdes_1') |(pines['Clase']== 'a_verdes_2')]
pines_verdes.shape

(440, 7)

In [96]:
cruce_terrenosxpines = gpd.sjoin(terrenos,pines_verdes)
cruce_terrenosxpines

,clase_dete,conf,area,geometry,status,con_const,CURT_f,index_right,tipoCenCom,tipo_asent,nombre_act,clee,ID_denue,Clase
5,terreno_baldio,0.21270,1900.646551,"POLYGON ((-11105776.741 2135138.249, -11105742...",NaN,1.0,18495364579945528538,15,NaN,NaN,NaN,NaN,NaN,a_verdes
8,terreno_baldio,0.62582,651.650246,"POLYGON ((-11105802.590 2135147.720, -11105767...",NaN,1.0,18495447759945538052,15,NaN,NaN,NaN,NaN,NaN,a_verdes
1174,terreno_baldio,0.51829,1415.931399,"POLYGON ((-11102238.948 2136390.358, -11102212...",NaN,0.0,18503238969943583595,74,NaN,NaN,NaN,NaN,NaN,a_verdes
1175,terreno_baldio,0.46722,2213.398444,"POLYGON ((-11102236.725 2136355.363, -11102169...",NaN,1.0,18503153679943578615,74,NaN,NaN,NaN,NaN,NaN,a_verdes
1446,terreno_baldio,0.20252,1572.810162,"POLYGON ((-11101929.989 2136077.392, -11101903...",NaN,0.0,18502284239943483087,88,NaN,NaN,NaN,NaN,NaN,a_verdes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,826,NaN,NaN,NaN,NaN,NaN,a_verdes
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,825,NaN,NaN,NaN,NaN,NaN,a_verdes
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,824,NaN,NaN,NaN,NaN,NaN,a_verdes
23855,terreno_baldio,0.35840,845.737434,"POLYGON ((-11095129.565 2134358.965, -11095128...",correcto,0.0,18493096589940087434,1295,NaN,NaN,NaN,NaN,NaN,a_verdes


In [97]:
cruce_terrenosxpines['PIN_si'] = 1
cruce_terrenosxpines

,clase_dete,conf,area,geometry,status,con_const,CURT_f,index_right,tipoCenCom,tipo_asent,nombre_act,clee,ID_denue,Clase,PIN_si
5,terreno_baldio,0.21270,1900.646551,"POLYGON ((-11105776.741 2135138.249, -11105742...",NaN,1.0,18495364579945528538,15,NaN,NaN,NaN,NaN,NaN,a_verdes,1
8,terreno_baldio,0.62582,651.650246,"POLYGON ((-11105802.590 2135147.720, -11105767...",NaN,1.0,18495447759945538052,15,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1174,terreno_baldio,0.51829,1415.931399,"POLYGON ((-11102238.948 2136390.358, -11102212...",NaN,0.0,18503238969943583595,74,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1175,terreno_baldio,0.46722,2213.398444,"POLYGON ((-11102236.725 2136355.363, -11102169...",NaN,1.0,18503153679943578615,74,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1446,terreno_baldio,0.20252,1572.810162,"POLYGON ((-11101929.989 2136077.392, -11101903...",NaN,0.0,18502284239943483087,88,NaN,NaN,NaN,NaN,NaN,a_verdes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,826,NaN,NaN,NaN,NaN,NaN,a_verdes,1
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,825,NaN,NaN,NaN,NaN,NaN,a_verdes,1
19776,terreno_baldio,0.30125,229.284346,"POLYGON ((-11095897.716 2136490.598, -11095850...",correcto,0.0,18503562399940333685,824,NaN,NaN,NaN,NaN,NaN,a_verdes,1
23855,terreno_baldio,0.35840,845.737434,"POLYGON ((-11095129.565 2134358.965, -11095128...",correcto,0.0,18493096589940087434,1295,NaN,NaN,NaN,NaN,NaN,a_verdes,1


In [98]:
no_cruzanxPines=terrenos[~(terrenos.geometry.isin(cruce_terrenosxpines['geometry']))]
no_cruzanxPines.shape

(10567, 7)

In [99]:
no_cruzanxPines['PIN_si']=0
no_cruzanxPines.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,clase_dete,conf,area,geometry,status,con_const,CURT_f,PIN_si
29193,terreno_baldio,0.26252,244.368842,"POLYGON ((-11088769.944 2136447.412, -11088761...",correcto,0.0,18503401949936432164,0
29199,terreno_baldio,0.23584,225.261813,"POLYGON ((-11088707.717 2136449.590, -11088699...",correcto,0.0,18503404499936412337,0


In [100]:
cruce_terrenosxpines.shape, no_cruzanxPines.shape

((88, 15), (10567, 8))

In [101]:
crucexpin=pd.concat([cruce_terrenosxpines,no_cruzanxPines])
crucexpin

,clase_dete,conf,area,geometry,status,con_const,CURT_f,index_right,tipoCenCom,tipo_asent,nombre_act,clee,ID_denue,Clase,PIN_si
5,terreno_baldio,0.21270,1900.646551,"POLYGON ((-11105776.741 2135138.249, -11105742...",NaN,1.0,18495364579945528538,15.0,NaN,NaN,NaN,NaN,NaN,a_verdes,1
8,terreno_baldio,0.62582,651.650246,"POLYGON ((-11105802.590 2135147.720, -11105767...",NaN,1.0,18495447759945538052,15.0,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1174,terreno_baldio,0.51829,1415.931399,"POLYGON ((-11102238.948 2136390.358, -11102212...",NaN,0.0,18503238969943583595,74.0,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1175,terreno_baldio,0.46722,2213.398444,"POLYGON ((-11102236.725 2136355.363, -11102169...",NaN,1.0,18503153679943578615,74.0,NaN,NaN,NaN,NaN,NaN,a_verdes,1
1446,terreno_baldio,0.20252,1572.810162,"POLYGON ((-11101929.989 2136077.392, -11101903...",NaN,0.0,18502284239943483087,88.0,NaN,NaN,NaN,NaN,NaN,a_verdes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29161,terreno_baldio,0.36052,497.788383,"POLYGON ((-11089033.483 2136406.426, -11089024...",correcto,0.0,18503243469936517728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
29174,terreno_baldio,0.22201,415.326469,"POLYGON ((-11089005.477 2134903.035, -11088933...",correcto,0.0,18494721729936500473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
29176,terreno_baldio,0.20070,693.886836,"POLYGON ((-11089008.374 2134902.995, -11088990...",correcto,0.0,18494648699936508885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
29193,terreno_baldio,0.26252,244.368842,"POLYGON ((-11088769.944 2136447.412, -11088761...",correcto,0.0,18503401949936432164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [102]:
crucexpin.rename(columns={'index_right':'ID_chin'},inplace=True)

In [103]:
crucexpin.columns

Index(['clase_dete', 'conf', 'area', 'geometry', 'status', 'con_const',
       'CURT_f', 'ID_chin', 'tipoCenCom', 'tipo_asent', 'nombre_act', 'clee',
       'ID_denue', 'Clase', 'PIN_si'],
      dtype='object')

In [34]:
#Lectura de base de zonas

zonas = gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\zonas_rurales_urbanas\15m.shp")
zonas.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA,geometry
160098,1512500010016004,15,125,0001,0016,004,Urbana,Típica,"POLYGON ((2807844.209 857424.049, 2807852.408 ..."
160099,1512500010016025,15,125,0001,0016,025,Urbana,Típica,"POLYGON ((2807780.538 857336.969, 2807799.790 ..."


In [104]:
zona_Ixta=zonas[zonas.CVE_MUN=='040']
# zona_Ixta.plot()

In [105]:
cruce_zona = gpd.sjoin(crucexpin,zona_Ixta.to_crs(3857),how='left')
cruce_zona.rename(columns={'index_right':'ID_15m'},inplace=True)
cruce_zona.shape, crucexpin.shape

((10971, 24), (10655, 15))

In [106]:
cruce_zona.columns

Index(['clase_dete', 'conf', 'area', 'geometry', 'status', 'con_const',
       'CURT_f', 'ID_chin', 'tipoCenCom', 'tipo_asent', 'nombre_act', 'clee',
       'ID_denue', 'Clase', 'PIN_si', 'ID_15m', 'CVEGEO', 'CVE_ENT', 'CVE_MUN',
       'CVE_LOC', 'CVE_AGEB', 'CVE_MZA', 'AMBITO', 'TIPOMZA'],
      dtype='object')

In [111]:
#Base de Areas verdes, camellones, glorietas
sia = gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\conjunto_de_datos\15sia.shp")
sia=sia.to_crs(3857)
sia=sia[sia.CVE_MUN=='040']
sia.tail(2)

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,AMBITO,geometry
5534,150400022,15,040,0022,En Operación,Escuela,Telesecundaria Libertad y Progreso,Secundaria,3,Rural,"POLYGON ((-11100760.771 2140646.357, -11100726..."
5535,150400022,15,040,0022,En Operación,Escuela,Telebachillerato Comunitario No. 78,Medio Superior,4,Rural,"POLYGON ((-11100756.458 2140653.334, -11100739..."


In [112]:
#Cruzar con ejes para quitar calles con overlay 
# Ejes de calles
ejes_path = r"H:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\DELEGACIONES_EDOMEX\IXTAPAN DE LA SAL\Ejes_Ixtapan_Sal.shp"
ejes = gpd.read_file(ejes_path)
ejes = ejes.to_crs(3857)
ejes.tail(2)

,STREETS,BANDA,geometry
2541,SIN NOMBRE,NaN,"LINESTRING (-11094855.669 2134653.725, -110948..."
2542,NaN,NaN,"LINESTRING (-11095010.198 2134729.794, -110949..."


In [113]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs ejes: ",ejes.crs)
print("crs sia: ",sia.crs)

crs ejes:  EPSG:3857
crs sia:  EPSG:3857


In [114]:
cruce_ejes = sia.sjoin(ejes)
print(cruce_ejes.shape)
'Porcentaje a borrar: ',cruce_ejes.shape[0]/sia.shape[0]*100

(55, 14)


('Porcentaje a borrar: ', 46.21848739495798)

In [115]:
import numpy as np 
print('Shape original: ', sia.shape)
index_drop = np.unique(sia[sia.geometry.isin(cruce_ejes.geometry)].index.values)
sia.drop(index=index_drop,inplace=True)
print('Shape final:    ', sia.shape)

Shape original:  (119, 11)
Shape final:     (89, 11)


In [116]:
cruce_zona.shape, sia.shape

((10971, 24), (89, 11))

*Cruce de terrenos vs areas verdes (sia)*

In [117]:
cruce_final = gpd.sjoin(cruce_zona,sia,how='left')
cruce_final.tail(2)

,clase_dete,conf,area,geometry,status,con_const,CURT_f,ID_chin,tipoCenCom,tipo_asent,...,CVEGEO_right,CVE_ENT_right,CVE_MUN_right,CVE_LOC_right,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,AMBITO_right
29193,terreno_baldio,0.26252,244.368842,"POLYGON ((-11088769.944 2136447.412, -11088761...",correcto,0.0,18503401949936432164,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29199,terreno_baldio,0.23584,225.261813,"POLYGON ((-11088707.717 2136449.590, -11088699...",correcto,0.0,18503404499936412337,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#Combinar las columnas que se duplican porque están en ambas bases y crear una general
cruce_final['CVEGEO']     =cruce_final.CVEGEO_left.combine_first(    cruce_final.CVEGEO_right)
cruce_final['CVE_ENT']    =cruce_final.CVE_ENT_left.combine_first(   cruce_final.CVE_ENT_right)
cruce_final['CVE_MUN']    =cruce_final.CVE_MUN_left.combine_first(   cruce_final.CVE_MUN_right)
cruce_final['CVE_LOC']    =cruce_final.CVE_LOC_left.combine_first(   cruce_final.CVE_LOC_right)
cruce_final['AMBITO']     =cruce_final.AMBITO_left.combine_first(    cruce_final.AMBITO_right)
cruce_final.rename(columns={'index_right':'ID_sia'},inplace=True)
#Eliminar las columnas divididas porque ya están en la general
cruce_final.drop(['CVEGEO_left','CVEGEO_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_ENT_left','CVE_ENT_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_MUN_left','CVE_MUN_right'],axis=1,inplace=True)
cruce_final.drop(['CVE_LOC_left','CVE_LOC_right'],axis=1,inplace=True)
cruce_final.drop(['AMBITO_left','AMBITO_right'],axis=1,inplace=True)

In [120]:
print('Columas de terrenos: ', cruce_final.columns)
if input('Quiere crear la columna ID_terreno') in ['1','si']:
    cruce_final.reset_index(drop=False, inplace=True)
    cruce_final.rename(columns={'index':'ID_terreno'},inplace=True)

Columas de terrenos:  Index(['clase_dete', 'conf', 'area', 'geometry', 'status', 'con_const',
       'CURT_f', 'ID_chin', 'tipoCenCom', 'tipo_asent', 'nombre_act', 'clee',
       'ID_denue', 'Clase', 'PIN_si', 'ID_15m', 'CVE_AGEB', 'CVE_MZA',
       'TIPOMZA', 'ID_sia', 'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO',
       'CVE_SERV', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'AMBITO'],
      dtype='object')


In [125]:
cruce_final.columns

Index(['ID_terreno', 'clase_dete', 'conf', 'area', 'geometry', 'status',
       'con_const', 'CURT_f', 'ID_chin', 'tipoCenCom', 'tipo_asent',
       'nombre_act', 'clee', 'ID_denue', 'Clase', 'PIN_si', 'ID_15m',
       'CVE_AGEB', 'CVE_MZA', 'TIPOMZA', 'ID_sia', 'CONDICION', 'GEOGRAFICO',
       'NOMSERV', 'TIPO', 'CVE_SERV', 'CVEGEO', 'CVE_ENT', 'CVE_MUN',
       'CVE_LOC', 'AMBITO'],
      dtype='object')

In [126]:
try:
    cruce_final = cruce_final[['ID_terreno','clase_dete','Clase', 'geometry','CURT_f','conf', 'area',  
                               'status', 'con_const','ID_chin', 'tipoCenCom', 'tipo_asent', 'nombre_act', 
                               'clee', 'ID_denue','PIN_si', 'ID_15m', 'CVE_AGEB', 'CVE_MZA', 'TIPOMZA', 'ID_sia',
                                'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO', 'CVE_SERV', 'CVEGEO',
                                'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'AMBITO']]
except:
    cruce_final = cruce_final[['ID_terreno','clase_dete','Clase', 'geometry','CURT_f','conf', 'area',  
                               'status', 'con_const','ID_chin', 'tipoCenCom', 'tipo_asent', 'nombre_act', 
                               'clee', 'ID_denue','PIN_si', 'ID_15m', 'CVE_AGEB', 'CVE_MZA', 'TIPOMZA', 'ID_sia',
                                'CONDICION', 'GEOGRAFICO', 'NOMSERV', 'TIPO', 'CVE_SERV', 'CVEGEO',
                                'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'AMBITO']]

In [127]:
cruce_final[cruce_final.ID_terreno.isna()]

,ID_terreno,clase_dete,Clase,geometry,CURT_f,conf,area,status,con_const,ID_chin,...,CONDICION,GEOGRAFICO,NOMSERV,TIPO,CVE_SERV,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,AMBITO


In [128]:
cruce_final.to_file(r'D:\Secretaria\cruces_bases\limpiezas_shp_base\Ixtapan\ultimas_versiones\Ixtapan_terrenos_cruces_todos.shp',
                 index=False)